# Exploratory Data Analysis with Pyspark and Spark SQL 

By Tom Urban and Ethan Smadja Gr3

The following notebook utilizes New York City taxi data from [TLC Trip Record Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

## Instructions

- Load and explore nyc taxi data from january 0f 2019. The exercises can be executed using pyspark or spark sql ( a subset of the questions will be re-answered using the language not chosen for the  main work).
- Load the zone lookup table to answer the questions about the nyc boroughs.  
- Load nyc taxi data from January of 2025 and compare data.  
- With any remaining time, work on the where to go from here section.  
- Lab due date is TBD ( due dates will be updated in the readme for the class repo )

In [0]:
# Define the name of the new catalog
catalog = 'taxi_eda_db'

# define variables for the trips data
schema = 'yellow_taxi_trips'
volume = 'data'
file_name = 'yellow_tripdata_2019-01.parquet'
table_name = 'tbl_yellow_taxi_trips'
path_volume = '/Volumes/' + catalog + "/" + schema + '/' + volume
path_table =  catalog + "." + schema
download_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-01.parquet'

In [0]:
# create the catalog/schema/volume
spark.sql('create catalog if not exists ' + catalog)
spark.sql('create schema if not exists ' + catalog + '.' + schema)
spark.sql('create volume if not exists ' + catalog + '.' + schema + '.' + volume)

In [0]:
# Get the data
dbutils.fs.cp(f"{download_url}", f"{path_volume}" + "/" + f"{file_name}")

In [0]:
# create the dataframe
df_trips = spark.read.parquet(f"{path_volume}/{file_name}",
  header=True,
  inferSchema=True,
  sep=",")

In [0]:
# Show the dataframe
df_trips.show()

## Lab

### Part 1
This section can be completed either using pyspark commands or sql commands ( There will be a section after in which a self-chosen subset of the questions are re-answered using the language not used for the main section. i.e. if pyspark is chosen for the main lab, sql should be used to repeat some of the questions. )

- Add a column that creates a unique key to identify each record in order to answer questions about individual trips
- Which trip has the highest passanger count
- What is the Average passanger count
- Shortest/longest trip by distance? by time?.
- busiest day/slowest single day
- busiest/slowest time of day ( you may want to bucket these by hour or create timess such as morning, afternoon, evening, late night )
- On average which day of the week is slowest/busiest
- Does trip distance or num passangers affect tip amount
- What was the highest "extra" charge and which trip
- Are there any datapoints that seem to be strange/outliers (make sure to explain your reasoning in a markdown cell)?

Step 1 : Add a column that creates a unique key to identify each record in order to answer questions about individual trips 

In [0]:
# let's import a function that generates a column with monotonically increasing 64-bit integers
from pyspark.sql.functions import monotonically_increasing_id

# here we create the column and apply the function
df_trips = df_trips.withColumn("trip_id", monotonically_increasing_id())

display(df_trips)

- Which trip has the highest passanger count ?


In [0]:
# the desc function returns a sort expression for the target column in descending order
from pyspark.sql.functions import desc

# let's use this function help to orderBy and display the first row wich correspond to the highest passanger count 
trip_max_passenger = df_trips.orderBy(desc("passenger_count")).limit(1).select("passenger_count", "trip_id")
display(trip_max_passenger)

- What is the Average passanger count


In [0]:
# Calculate the average passenger count
from pyspark.sql.functions import avg

# Compute the average and add a comment to the result column
avg_passenger_count = df_trips.agg(avg("passenger_count").alias("average_passenger_count"))

display(avg_passenger_count)

- Shortest/longest trip by distance? by time?.


In [0]:
from pyspark.sql.functions import col, unix_timestamp, min as spark_min, max as spark_max

# Calculate trip duration in seconds
df_trips = df_trips.withColumn(
    "trip_duration_sec",
    unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")
)

# Shortest and longest trip by distance
shortest_trip_distance = df_trips.orderBy(
    col("trip_distance").asc()
).limit(1).select("trip_distance", "trip_id")

longest_trip_distance = df_trips.orderBy(
    col("trip_distance").desc()
).limit(1).select("trip_distance", "trip_id")

# Shortest and longest trip by duration
shortest_trip_time = df_trips.orderBy(
    col("trip_duration_sec").asc()
).limit(1).select("trip_duration_sec", "trip_id")

longest_trip_time = df_trips.orderBy(
    col("trip_duration_sec").desc()
).limit(1).select("trip_duration_sec", "trip_id")

#  display the results with .show()
print("the shortest trip by distance is :")
shortest_trip_distance.show()

print("the longest trip by distance is :")
longest_trip_distance.show()

print("the shortest trip by time is :")
shortest_trip_time.show()

print("the longest trip by time is:")
longest_trip_time.show()

we can see here that there are outliers values in the dataset since there is negative trip duration value


- Busiest / Slowest single day


In [0]:
from pyspark.sql.functions import to_date, col, count

# Extract pickup date
df_trips = df_trips.withColumn("pickup_date", to_date(col("tpep_pickup_datetime")))

# Count trips per day
trips_per_day = df_trips.groupBy("pickup_date").agg(count("*").alias("num_trips"))

# Busiest and slowest days
busiest_day = trips_per_day.orderBy(col("num_trips").desc()).limit(1)
slowest_day = trips_per_day.orderBy(col("num_trips").asc()).limit(1)

busiest_day.show()
slowest_day.show()

- Busiest / Slowest time of day

In [0]:
from pyspark.sql.functions import hour

# Extract the hour from the pickup datetime
df_trips = df_trips.withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))

# Count how many trips started in each hour
trips_per_hour = df_trips.groupBy("pickup_hour").agg(count("*").alias("num_trips"))

# Find the busiest and slowest hours
busiest_hour = trips_per_hour.orderBy(col("num_trips").desc()).limit(1)
slowest_hour = trips_per_hour.orderBy(col("num_trips").asc()).limit(1)

# Show the results
busiest_hour.show()
slowest_hour.show()


- Find the busiest and slowest day of the week (on average)

In [0]:
from pyspark.sql.functions import dayofweek

# Extract the day of the week from the pickup date
df_trips = df_trips.withColumn("day_of_week", dayofweek(col("pickup_date")))

# Count total trips for each day of the week
avg_trips_by_day = df_trips.groupBy("day_of_week").agg(count("*").alias("num_trips"))

# Show which day of the week is the busiest and slowest
avg_trips_by_day.orderBy(col("num_trips").desc()).show()


- Check if distance or passengers affect the tip amount

In [0]:
# Calculate correlation between trip distance and tip amount
print("Correlation (distance vs tip):", df_trips.stat.corr("trip_distance", "tip_amount"))

# Calculate correlation between passenger count and tip amount
print("Correlation (passengers vs tip):", df_trips.stat.corr("passenger_count", "tip_amount"))

# Show average tip amount by passenger count
from pyspark.sql.functions import avg

df_trips.groupBy("passenger_count").agg(avg("tip_amount").alias("avg_tip")).orderBy("passenger_count").show()

- What was the highest "extra" charge and which trip


In [0]:
from pyspark.sql.functions import max

# Find the maximum extra charge
max_extra = df_trips.agg(max("extra").alias("max_extra")).collect()[0]["max_extra"]

# Show the trip(s) that had this highest extra charge
df_trips.filter(col("extra") == max_extra).select("trip_id", "extra", "tpep_pickup_datetime", "trip_distance").show()

- Are there any datapoints that seem to be strange/outliers (make sure to explain your reasoning in a markdown cell)?

In [0]:
df_trips.filter(
    (col("trip_distance") <= 0) |      # distance cannot be 0 or negative
    (col("fare_amount") < 0) |         # fare cannot be negative
    (col("passenger_count") > 6)       # more than 6 passengers is unusual
).show()

Some trips show unrealistic values such as a distance of 0 or negative, negative fares, or more than six passengers.
These data points are likely caused by data entry or recording errors and should be treated as outliers.


### Part 2

- Using the code for loading the first dataset as an example, load in the taxi zone lookup and answer the following questions
- which borough had most pickups? dropoffs?
- what are the busy/slow times by borough 
- what are the busiest days of the week by borough?
- what is the average trip distance by borough?
- what is the average trip fare by borough?
- highest/lowest faire amounts for a trip, what burough is associated with the each
- load the dataset from the most recently available january, is there a change to any of the average metrics.

Load the taxi zone lookup dataset


In [0]:


# Define variables for the taxi zone lookup file
zone_schema = 'taxi_zone_lookup'
zone_volume = 'data'
zone_file_name = 'taxi+_zone_lookup.csv'
zone_table_name = 'tbl_taxi_zone_lookup'
zone_path_volume = '/Volumes/' + catalog + "/" + zone_schema + '/' + zone_volume
zone_path_table = catalog + "." + zone_schema
zone_download_url = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv'

# Create the schema and volume if they don't exist
spark.sql('create schema if not exists ' + catalog + '.' + zone_schema)
spark.sql('create volume if not exists ' + catalog + '.' + zone_schema + '.' + zone_volume)

# Download the lookup CSV file into your volume
dbutils.fs.cp(f"{zone_download_url}", f"{zone_path_volume}/{zone_file_name}")

# Load the lookup table into a Spark DataFrame
df_zones = spark.read.csv(f"{zone_path_volume}/{zone_file_name}", header=True, inferSchema=True)

df_zones.show(5)


Join trips with zone lookup


In [0]:

# Join for pickup borough
df_trips = df_trips.join(
    df_zones.withColumnRenamed("LocationID", "PULocationID_lookup"),
    df_trips["PULocationID"] == col("PULocationID_lookup"),
    "left"
).withColumnRenamed("Borough", "pickup_borough")

# Join for dropoff borough
df_trips = df_trips.join(
    df_zones.withColumnRenamed("LocationID", "DOLocationID_lookup"),
    df_trips["DOLocationID"] == col("DOLocationID_lookup"),
    "left"
).withColumnRenamed("Borough", "dropoff_borough")


- which borough had most pickups? dropoffs?


In [0]:

from pyspark.sql.functions import count

# Count pickups per borough
pickup_counts = df_trips.groupBy("pickup_borough").agg(count("*").alias("num_pickups")).orderBy(col("num_pickups").desc())
pickup_counts.show()

# Count dropoffs per borough
dropoff_counts = df_trips.groupBy("dropoff_borough").agg(count("*").alias("num_dropoffs")).orderBy(col("num_dropoffs").desc())
dropoff_counts.show()


- what are the busy/slow times by borough 


In [0]:

# Group by pickup_borough and hour
trips_by_borough_hour = df_trips.groupBy("pickup_borough", "pickup_hour").agg(count("*").alias("num_trips"))

# Show the busiest hours for each borough
trips_by_borough_hour.orderBy(col("pickup_borough"), col("num_trips").desc()).show()


- what are the busiest days of the week by borough?


In [0]:

trips_by_borough_day = df_trips.groupBy("pickup_borough", "day_of_week").agg(count("*").alias("num_trips"))

# Show results
trips_by_borough_day.orderBy(col("pickup_borough"), col("num_trips").desc()).show()


- highest/lowest fare amounts for a trip, what borough is associated with the each


In [0]:


from pyspark.sql.functions import max, min

# Highest fare by borough
max_fares = df_trips.groupBy("pickup_borough").agg(max("fare_amount").alias("max_fare"))
max_fares.show()

# Lowest fare by borough
min_fares = df_trips.groupBy("pickup_borough").agg(min("fare_amount").alias("min_fare"))
min_fares.show()


- what is the average trip distance by borough?


In [0]:
from pyspark.sql.functions import avg

# Average distance per pickup borough
avg_distance = df_trips.groupBy("pickup_borough").agg(avg("trip_distance").alias("avg_distance"))
avg_distance.show()

- what is the average trip fare by borough?


In [0]:

from pyspark.sql.functions import avg

# Average fare per pickup borough
avg_fare = df_trips.groupBy("pickup_borough").agg(avg("fare_amount").alias("avg_fare"))
avg_fare.show()


- load the dataset from the most recently available january, is there a change to any of the average metrics.


In [0]:

latest_file = 'yellow_tripdata_2025-01.parquet'
latest_download_url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{latest_file}'

# Copy and load the new data
dbutils.fs.cp(latest_download_url, f"{path_volume}/{latest_file}")

# Read the latest dataset
df_trips_latest = spark.read.parquet(f"{path_volume}/{latest_file}", header=True, inferSchema=True)



# Join with the taxi zone lookup to get borough names
# Join for pickup borough
df_trips_latest = df_trips_latest.join(
    df_zones.withColumnRenamed("LocationID", "PULocationID_lookup"),
    df_trips_latest["PULocationID"] == col("PULocationID_lookup"),
    "left"
).withColumnRenamed("Borough", "pickup_borough")
  
#Average trip distance per pickup borough
avg_distance_latest = (
    df_trips_latest.groupBy("pickup_borough")
    .agg(avg("trip_distance").alias("avg_trip_distance"))
    .orderBy(col("avg_trip_distance").desc())
)

print("Average Trip Distance by Borough (January 2025):")
avg_distance_latest.show()

# Average fare amount per pickup borough
avg_fare_latest = (
    df_trips_latest.groupBy("pickup_borough")
    .agg(avg("fare_amount").alias("avg_fare_amount"))
    .orderBy(col("avg_fare_amount").desc())
)

print("Average Fare Amount by Borough (January 2025):")
avg_fare_latest.show()


- **Trip distances** increased significantly in 2025, especially in the **Bronx** and **Brooklyn**, which may suggest longer routes or data recording changes.  
- **Fares** generally rose, especially for **Queens** and **EWR (Newark Airport)**, possibly reflecting higher fuel costs or fare adjustments.  
- **Manhattan** still has the **shortest and cheapest trips**, consistent with its dense urban layout.  
- **Staten Island** shows a notable **drop in fare and distance**, possibly due to fewer long-distance trips or outlier filtering differences.  
- The **N/A and Unknown** categories likely represent incomplete location data and should be interpreted cautiously.


### Part 3

- choose 3 questions from above and re-answer them using the language you did not use for the main notebook . (i.e - if you completed the exercise in python, redo 3 questions in pure sql) . at least one of the questions to be redone must involve a join

In [0]:
# Register the DataFrame as a temporary view for SQL queries
df_trips_latest.createOrReplaceTempView("yellow_trips_latest")
df_zones.createOrReplaceTempView("taxi_zones")


Busy/slow times by borough (hour)

In [0]:
%sql
SELECT
    pickup_borough,
    HOUR(tpep_pickup_datetime) AS pickup_hour,
    COUNT(*) AS trip_count
FROM yellow_trips_latest
WHERE pickup_borough IS NOT NULL
GROUP BY pickup_borough, HOUR(tpep_pickup_datetime)
ORDER BY pickup_borough, trip_count DESC;


Busiest days of the week by borough (with JOIN)

In [0]:
%sql
SELECT
    z.Borough AS pickup_borough,
    date_format(t.tpep_pickup_datetime, 'E') AS day_of_week,
    COUNT(*) AS trip_count
FROM yellow_trips_latest AS t
LEFT JOIN taxi_zones AS z
    ON t.PULocationID = z.LocationID
WHERE z.Borough IS NOT NULL
GROUP BY z.Borough, date_format(t.tpep_pickup_datetime, 'E')
ORDER BY z.Borough, trip_count DESC;


Average trip distance by borough

In [0]:
%sql
SELECT
    pickup_borough,
    ROUND(AVG(trip_distance), 2) AS avg_trip_distance
FROM yellow_trips_latest
WHERE pickup_borough IS NOT NULL
GROUP BY pickup_borough
ORDER BY avg_trip_distance DESC;



### Part 4

As of spark v4 dataframes have native visualization support. Choose at least 3 questions from above and provide visualizations.


In [0]:
from pyspark.sql.functions import avg, col

# Calculate average trip distance per pickup borough
avg_distance_by_borough = (
    df_trips_latest.groupBy("pickup_borough")
    .agg(avg("trip_distance").alias("avg_trip_distance"))
    .orderBy(col("avg_trip_distance").desc())
)


display(avg_distance_by_borough) 


Databricks visualization. Run in Databricks to view.

In [0]:
# Calculate average fare per pickup borough
avg_fare_by_borough = (
    df_trips_latest.groupBy("pickup_borough")
    .agg(avg("fare_amount").alias("avg_fare"))
    .orderBy(col("avg_fare").desc())
)


display(avg_fare_by_borough)  


Databricks visualization. Run in Databricks to view.

In [0]:
# Calculate average fare per pickup borough
avg_fare_by_borough = (
    df_trips_latest.groupBy("pickup_borough")
    .agg(avg("fare_amount").alias("avg_fare"))
    .orderBy(col("avg_fare").desc())
)

display(avg_fare_by_borough) 


Databricks visualization. Run in Databricks to view.

# Where to go from here

- Continue building the dataset by loading in more data, start by completing the data for 2019 and calculating the busiest season (fall, winter, spring, summer)
- Explore a dataset/datasets of your choosing

In [0]:
import pyspark.sql.functions as F

catalog = "taxi_eda_db"
schema = "yellow_taxi_trips"
volume = "data"
path_volume = f"/Volumes/{catalog}/{schema}/{volume}"

months = [f"{m:02d}" for m in range(1, 13)]

# Initialize an empty DataFrame
df_2019 = None

for m in months:
    file_name = f"yellow_tripdata_2019-{m}.parquet"
    download_url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file_name}"
    target_path = f"{path_volume}/{file_name}"

    
    df_2019 = df_month if df_2019 is None else df_2019.union(df_month)
    
print("dataset fully loaded")




Add a “Season” Column

We can define seasons by month:

Winter: Dec–Feb (12, 1, 2)

Spring: Mar–May (3, 4, 5)

Summer: Jun–Aug (6, 7, 8)

Fall: Sep–Nov (9, 10, 11)


In [0]:
from pyspark.sql.functions import month, when, count

df_2019 = df_2019.withColumn("month", month("tpep_pickup_datetime"))

df_2019 = df_2019.withColumn(
    "season",
    when(F.col("month").isin(12, 1, 2), "Winter")
    .when(F.col("month").isin(3, 4, 5), "Spring")
    .when(F.col("month").isin(6, 7, 8), "Summer")
    .otherwise("Fall")
)

# Find busiest season
busiest_season = (
    df_2019.groupBy("season")
    .agg(count("*").alias("trip_count"))
    .orderBy(F.col("trip_count").desc())
)

display(busiest_season)


Spotify Tracks Dataset — PySpark Analysis

Dataset: *Ultimate Spotify Tracks DB* (Kaggle)

#### Key Transformations
- Converted Kaggle dataset from Pandas → Spark DataFrame.
- Added derived columns (`energy_level`, `mood`, `loudness_level`).
- Grouped and aggregated data by genre, mood, and energy.




In [0]:
import kagglehub
import pandas as pd

# Télécharge le dataset
path = kagglehub.dataset_download("zaheenhamidani/ultimate-spotify-tracks-db")

# Vérifie les fichiers téléchargés
import os
print(os.listdir(path))


In [0]:
# Load the CSV into a pandas dataframe
file_path = os.path.join(path, "SpotifyFeatures.csv")
df_spotify_pd = pd.read_csv(file_path)


print(df_spotify_pd.shape)
df_spotify_pd.head()


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, when, round, stddev

# Create Spark session (if not already running)
spark = SparkSession.builder.getOrCreate()

# Convert pandas DataFrame → Spark DataFrame
df_spotify = spark.createDataFrame(df_spotify_pd)

# Register a temporary SQL view for SparkSQL (optional)
df_spotify.createOrReplaceTempView("spotify_tracks")

# Show sample rows & schema
df_spotify.show(5)
df_spotify.printSchema()


Let's see the average popularity by genre 

In [0]:
avg_popularity_by_genre = (
    df_spotify.groupBy("genre")
    .agg(
        round(avg("popularity"), 2).alias("avg_popularity"),
        count("*").alias("num_tracks")
    )
    .filter(col("num_tracks") > 50)
    .orderBy(col("avg_popularity").desc())
)

display(avg_popularity_by_genre)

Databricks visualization. Run in Databricks to view.

Average Danceability by Genre


In [0]:
danceability_by_genre = (
    df_spotify.groupBy("genre")
    .agg(round(avg("danceability"), 2).alias("avg_danceability"))
    .orderBy(col("avg_danceability").desc())
)

display(danceability_by_genre)


Databricks visualization. Run in Databricks to view.

Mood Classification (Energy + Valence)

In [0]:
df_spotify_mood = df_spotify.withColumn(
    "mood",
    when((col("energy") > 0.6) & (col("valence") > 0.6), "Happy / Energetic")
    .when((col("energy") < 0.4) & (col("valence") < 0.4), "Sad / Calm")
    .when((col("energy") > 0.6) & (col("valence") < 0.4), "Angry / Intense")
    .otherwise("Neutral")
)

mood_summary = (
    df_spotify_mood.groupBy("mood")
    .agg(
        count("*").alias("num_tracks"),
        round(avg("popularity"), 2).alias("avg_popularity")
    )
    .orderBy(col("num_tracks").desc())
)

display(mood_summary)


Databricks visualization. Run in Databricks to view.